# Genaction: Data Visualization Pipeline
This is a playground pipeline that collects from multiple data sources. Not all code is final. To see the final code, feel free to take a look at [this repo](https://github.com/SteamlabsCA/data-visualization)

# Installing Dependencies

In [1]:
!pip3 install xlsxwriter pandas xarray bs4 requests lxml

# Importing Dependencies

In [2]:
from datetime import date, datetime
from io import BytesIO
from zipfile import ZipFile

import pandas as pd
import numpy as np
import requests as r
from bs4 import BeautifulSoup
import xarray as xr
import os

import glob
import shutil
import pathlib

# Configuring Settings

## Setting Path

In [3]:
path = 'data'

## Pandas Settings

In [4]:
pd.set_option('display.max_columns', None)

# Hourly Fuel Output

## Downloading Data Utilities

### Extract Ontario Fuel Output
At the moment, I'm aggregating historical energy output in Ontario. The data is split into two parts.

1. `PUB_GenOutputbyFuelHourly.xml`: Hourly energy usage output from
the previous 12 months.

2. `PUB_GenOutputbyFuelHourly_XXXX.xml`: Hourly energy usage output
from year XXXX.

However, the format of the dataset is consistent. It's as follows:

```xml
<DocBody>
    <DeliveryYear>2022</DeliveryYear>
    <DailyData>
        <Day>2022-01-01</Day>
        <HourlyData>
            <Hour>1</Hour>

            <FuelTotal>
                <Fuel>NUCLEAR</Fuel>
                <EnergyValue>
                    <OutputQuality>0</OutputQuality>
                    <Output>10474</Output>
                </EnergyValue>
            </FuelTotal>
            [...]

        </HourlyData>
        [...]
    </DailyData>
    [...]
</DocBody>
```

Note that `[...]` implies repeated elements.

Collecting the data is a matter of traversing the tree and collecting
the needed information.

One issue I've run into is that BeautifulSoup takes a few seconds to
parse the needed XML. However, I will ignore this as it doesn't cause
significant problems.

In the situation where this may cause issues later, I would switch to
an XML parsing library such as `lxml`.

## Transforming Data Utilities

In [26]:
def _extract_fuel_output(url):
  xml = r.get(url).content

  soup = BeautifulSoup(xml, "xml")

  year = soup.find("DeliveryYear").text

  daily_data = soup.find_all("DailyData")

  for day_data in daily_data:
    date = day_data.find("Day")
    if date:
      date_obj = datetime.strptime(date.text, '%Y-%m-%d')

      month = date_obj.month
      day = date_obj.day

    hourly_datas = day_data.find_all("HourlyData")

    for hourly_data in hourly_datas:
      hour = hourly_data.find("Hour")
      if hour:
        hour = int(hour.text)

      fuel_totals = hourly_data.find_all("FuelTotal")

      for fuel_total in fuel_totals:
        fuel = fuel_total.find("Fuel")
        if fuel:
          fuel = fuel.text

        output = fuel_total.find("Output")
        if output:
          output = int(output.text)

        yield {
          "year": year,
          "month": month,
          "day": day,
          "hour": hour,
          "fuel": fuel,
          "output": output
        }

### Get Provincial/National Fuel Output Percentages

In [14]:
def _collect_fuel_output_percentage(YEAR_TO_COLLECT):
  fuel_output_percentages = []

  regions = pd \
    .read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2021/electricity-generation-2021.csv") \
    .query(f"Year == {YEAR_TO_COLLECT}") \
    .query(f"Scenario == 'Current Policies'") \
    .drop(["Scenario", "Type", "Type", "Unit"], axis=1) \
    .groupby("Region")

  for region_name, region_df in regions:
    total_output = region_df["Value"].sum().round(2)

    baseload_output = region_df.query("Variable in ('NUCLEAR', 'HYDRO', 'BIOFUEL')")["Value"].sum()
    solar_output = region_df.query("Variable == 'Solar'").iloc[0]["Value"]
    wind_output = region_df.query("Variable == 'Wind'").iloc[0]["Value"]

    baseload_percent = baseload_output / total_output
    solar_percent = solar_output / total_output
    wind_percent = wind_output / total_output

    fuel_output_percentages.append({
      "region_name": region_name,
      "baseload_percent": baseload_percent,
      "solar_percent": solar_percent,
      "wind_percent": wind_percent
    })

    return fuel_output_percentages

### Scaling Ontario Fuel Output

In [15]:
def _transform_ontario_fuel_output(
    fuel_output_df,
    provincial_baseload_percent,
    provincial_solar_percent,
    provincial_wind_percent
  ):
  """Step 0 - Setting Needed Values"""
  # 24 hour clock
  # See Step 1.1 for application.
  time_blocks = [
      (7, 8, 9),
      (11, 12, 13),
      (15, 16, 17, 18, 19),
      (21, 22, 23, 24, 1),
      (3, 4, 5)
  ]

  """Step 1 - Basic Preprocessing"""
  """
  Step 1.1 - Apply the following transformations to the fuel output
    * 1.1.1
    * 1.1.2. Group by fuel type
    * 1.1.3. Group by month
    * 1.1.4. Group by time blocks (see constants)
    * 1.1.5. Calculate the mean of every time block

  It should possible for a user to get an item from the final DataFrame
  by the fuel type, month and time.

  For example, a user can get solar energy output from January at 1am
  """

  data = []

  grouped_by_year = fuel_output_df.groupby("year") # 1.1.1

  for _, date_df in grouped_by_year:

    grouped_by_fuel_type = date_df.groupby("fuel") # 1.1.2

    for _, fuel_type_df in grouped_by_fuel_type:
      grouped_by_month = fuel_type_df.groupby("month") # 1.1.3

      for _, month_df in grouped_by_month:

        for time_block in time_blocks:
          grouped_by_time_block = month_df[month_df["hour"].isin(time_block)] # 1.1.4

          year = grouped_by_time_block["year"].iloc[0]
          month = grouped_by_time_block["month"].iloc[0]
          start_hour = time_block[0]
          end_hour = time_block[-1]
          fuel = grouped_by_time_block["fuel"].iloc[0]
          output = grouped_by_time_block["output"].mean().round(2) # 1.1.5

          data.append(
            {
                "year": year,
                "month": month,
                "start_hour": start_hour,
                "end_hour": end_hour,
                "fuel": fuel,
                "output": output
            }
          )

  fuel_output_df = pd.DataFrame(data)

  """Step 1.2 - Getting arrays of the power generation values"""
  gas = fuel_output_df[fuel_output_df["fuel"] == "GAS"] \
    .reset_index() \
    .drop(["index"], axis=1)

  wind = fuel_output_df[fuel_output_df["fuel"] == "WIND"] \
    .reset_index() \
    .drop(["index"], axis=1)

  solar = fuel_output_df[fuel_output_df["fuel"] == "SOLAR"] \
    .reset_index() \
    .drop(["index"], axis=1)

  # Baseload Fuels
  nuclear = fuel_output_df[fuel_output_df["fuel"] == "NUCLEAR"] \
    .reset_index() \
    .drop(["index"], axis=1)

  hydro = fuel_output_df[fuel_output_df["fuel"] == "HYDRO"] \
    .reset_index() \
    .drop(["index"], axis=1)

  biofuel = fuel_output_df[fuel_output_df["fuel"] == "BIOFUEL"] \
    .reset_index() \
    .drop(["index"], axis=1)

  """Step 1.3 - Getting the total power array"""
  total_energy_generated = gas["output"] + wind["output"] + solar["output"] + nuclear["output"] + hydro["output"] + biofuel["output"]

  """Step 2 - Getting Scaling Factors"""
  """Step 2.1 - Getting the percent ratio between Alberta's baseload percent and Ontario's baseload percent"""
  ONTARIO_BASELOAD_PERCENT = 0.83
  baseload_percent_ratio = provincial_baseload_percent/ONTARIO_BASELOAD_PERCENT

  """Step 2.2 - Getting the percent ratio between Alberta's solar percent and Ontario's solar percent"""
  ONTARIO_SOLAR_PERCENT = 0.01
  solar_percent_ratio = provincial_solar_percent/ONTARIO_SOLAR_PERCENT

  """Step 2.3 - Getting the percent ratio between Alberta's wind percent and Ontario's wind percent"""
  ONTARIO_WIND_PERCENT = 0.08
  wind_percent_ratio = provincial_wind_percent/ONTARIO_WIND_PERCENT

  """Step 3 - Scaling the Power"""
  """Step 3.1 - Multiply each item in the solar power array by the solar percent ratio"""
  solar["output_scaled"] = solar["output"] * solar_percent_ratio

  """Step 3.2 - Multiply each item in the wind power array by the wind percent ratio"""
  wind["output_scaled"] = wind["output"] * wind_percent_ratio

  """Step 3.3 - Multiply each item in the baseload power array by the baseload percent ratio"""
  nuclear["output_scaled"] = nuclear["output"] * baseload_percent_ratio
  hydro["output_scaled"] = hydro["output"] * baseload_percent_ratio
  biofuel["output_scaled"] = biofuel["output"] * baseload_percent_ratio
  gas["output_scaled"] = gas["output"] * baseload_percent_ratio

  """Step 3.4 - Add each item from the scaled solar, scaled wind, and scaled baseload arrays"""
  total_energy_generated_scaled = solar["output_scaled"] + wind["output_scaled"] + nuclear["output_scaled"] + hydro["output_scaled"] + biofuel["output_scaled"] + gas["output_scaled"]

  """Step 3.5 - Subtract the above value from each item in the total list"""
  fossil_fuels_scaled = total_energy_generated - total_energy_generated_scaled
  fossil_fuels_scaled = fossil_fuels_scaled.round(2)

  """Step 4 - Power Array Percentages"""
  """Step 4.1 - Divide each of the scaled arrays by the total list"""
  # Note, in Linear Algebra, it involves taking the inverse of the total list and
  # multiplying it by every scaled array.
  solar["power_percentage"] = solar["output_scaled"] / total_energy_generated_scaled
  wind["power_percentage"] = wind["output_scaled"] / total_energy_generated_scaled
  nuclear["power_percentage"] = nuclear["output_scaled"] / total_energy_generated_scaled
  hydro["power_percentage"] = hydro["output_scaled"] / total_energy_generated_scaled
  biofuel["power_percentage"] = biofuel["output_scaled"] / total_energy_generated_scaled
  gas["power_percentage"] = gas["output_scaled"] / total_energy_generated_scaled

  """Step 4.2 - Multiply by 100 to get total power percentage"""
  # Note that some outputs are so absurdly small that you'd spend 30 minutes trying
  # to multiply by 10,000 thinking that Pandas is weird when in reality... they're
  # just REALLY small.
  solar["power_percentage"] *= 100
  wind["power_percentage"] *= 100
  nuclear["power_percentage"] *= 100
  hydro["power_percentage"] *= 100
  biofuel["power_percentage"] *= 100
  gas["power_percentage"] *= 100

  """Step 5 - Final Clean"""
  """Step 5.1 - Round values over 100"""
  solar[solar["power_percentage"] > 100] = 100
  wind[wind["power_percentage"] > 100] = 100
  nuclear[nuclear["power_percentage"] < 0] = 0
  hydro[hydro["power_percentage"] < 0] = 0
  biofuel[biofuel["power_percentage"] < 0] = 0
  gas[gas["power_percentage"] < 0] = 0

  """Step 5.2 - Round values under 0"""
  solar[solar["power_percentage"] > 100] = 100
  wind[wind["power_percentage"] > 100] = 100
  nuclear[nuclear["power_percentage"] < 0] = 0
  hydro[hydro["power_percentage"] < 0] = 0
  biofuel[biofuel["power_percentage"] < 0] = 0
  gas[gas["power_percentage"] < 0] = 0

  return solar, wind, nuclear, hydro, biofuel, gas, fossil_fuels_scaled

## IESO

In [28]:
def _extract_hourly_fuel_output():
  '''Extracts hourly fuel output data'''
  fuel_output_hourly_url = "http://reports.ieso.ca/public/GenOutputbyFuelHourly/PUB_GenOutputbyFuelHourly.xml"
  fuel_output_hourly = list(_extract_fuel_output(fuel_output_hourly_url))

  return fuel_output_hourly

def _transform_hourly_fuel_output():
  hourly_fuel_output = _extract_hourly_fuel_output()
  hourly_fuel_output_df = pd.DataFrame(hourly_fuel_output)

  solar, wind, nuclear, hydro, biofuel, gas, fossil_fuels_scaled = _transform_ontario_fuel_output(hourly_fuel_output_df, 0.0, 0.005, 0.06)

  total_fuel_output = pd.concat(
    {
      "Year": solar["year"],
      "Month": solar["month"],
      "Start Hour": solar["start_hour"],
      "End Hour": solar["end_hour"],
      "Solar Output": solar["output"],
      "Wind Output": wind["output"],
      "Nuclear Output": nuclear["output"],
      "Hydro Output": hydro["output"],
      "Biofuel Output": biofuel["output"],
      "Gas Output": gas["output"],
      "Fossil Fuels Scaled": fossil_fuels_scaled
    },
    axis=1
  )

  total_fuel_output["Total Output"] = total_fuel_output \
    [["Solar Output", "Wind Output", "Nuclear Output", "Hydro Output", "Biofuel Output", "Gas Output", "Fossil Fuels Scaled"]] \
    .sum(axis=1) \
    .round(2)

  return total_fuel_output

def _load_hourly_fuel_output():
  total_fuel_output = _transform_hourly_fuel_output()

  csv_path = f"{path}/hourly_fuel_output/Hourly Fuel Output Ontario - IESO.csv"
  xlsx_path = f"{path}/hourly_fuel_output/Hourly Fuel Output Ontario - IESO.xlsx"

  total_fuel_output.to_csv(csv_path)
  total_fuel_output.to_excel(xlsx_path)

_load_hourly_fuel_output()

/home/t/code/steamlabs-2023/gen-content/venv/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'text'

# Mean Temperature

## Downloading Data Utilities

### Climate Atlas Of Canada

I'm not proud of this method... however, I find it as an easier temporary solution...

Rather then download and query data from the PCIC, I will directly download data from the Climate Atlas of Canada.

I don't like doing this. However, given the time constraints, I certaintly need to get this done ASAP.

A major TODO is to use PCIC data to collect the following data. I don't think it's right to scrape data that someone else preprocessed.

In [ ]:
def _download_climate_atlas_data(city_num: int, variable: str):
  possible_values = {"precip", "meantemp"}
  if not variable in possible_values:
    raise ValueError(f"Invalid `variable` value. Should be in {possible_values}")
  
  url = f"https://data.climateatlas.ca/csv/city_{city_num}/BCCAQv2/85/city_{city_num}_BCCAQv2_RCP85_annual_{variable}_ensemble.csv"

  if r.get(url).status_code == 404:
    raise ValueError("Invalid `city_num` value.")

  # The first 7 lines of any Climate Atlas Data Sheet looks
  # like the following (for example)
  #
  # 1 | Bias  correction  method:  BCCAQv2
  # 2 | Model  name:  Ensemble
  # 3 | RCP:  85
  # 4 | Variable:  tasmax  and  tasmin
  # 5 | Season:  annual
  # 6 | 1950-01-01  to  2095-12-31
  # 7 | city_445:  Average  value  Mean  Ensemble
  #
  # I will skip these lines in order to get the needed
  # data
  return pd.read_csv(url, skiprows=7)

## Climate Atlas Of Canada

In [ ]:
# TODO: Specify the ID as a part of the data from the Climate Atlas of Canada.
# the rough part would be tediously renaming everything. But alas, it would
# clarify the distinction between the various data sources.
cities = [
  { "City Name": "Timmins", "Province Name": "Ontario", "ID": "430" },
  { "City Name": "Ottawa", "Province Name": "Ontario", "ID": "459" },
  { "City Name": "Toronto", "Province Name": "Ontario", "ID": "458" },
  { "City Name": "Thunder Bay", "Province Name": "Ontario", "ID": "444" },
  { "City Name": "Regina", "Province Name": "Saskatchewan", "ID": "457" },
  { "City Name": "Saskatoon", "Province Name": "Saskatchewan", "ID": "445" },
  { "City Name": "Thompson", "Province Name": "Manitoba", "ID": "343" },
  { "City Name": "Brandon", "Province Name": "Manitoba", "ID": "430" },
  { "City Name": "Winnipeg", "Province Name": "Manitoba", "ID": "465" },
  { "City Name": "Prince George", "Province Name": "British Columbia", "ID": "443" },
  { "City Name": "Kelowna", "Province Name": "British Columbia", "ID": "442" },
  { "City Name": "Vancouver", "Province Name": "British Columbia", "ID": "455" },
  { "City Name": "Grande Prairie", "Province Name": "Alberta", "ID": "436" },
  { "City Name": "Edmonton", "Province Name": "Alberta", "ID": "466" },
  { "City Name": "Fort McMurray", "Province Name": "Alberta", "ID": "437" },
  { "City Name": "Calgary", "Province Name": "Alberta", "ID": "454" },
  { "City Name": "Edmundston", "Province Name": "New Brunswick", "ID": "273" },
  { "City Name": "Bathurst", "Province Name": "New Brunswick", "ID": "274" },
  { "City Name": "Frecdricton", "Province Name": "New Brunswick", "ID": "461" },
  { "City Name": "Moncton", "Province Name": "New Brunswick", "ID": "394" },
  { "City Name": "Saint John", "Province Name": "New Brunswick", "ID": "393" },
  { "City Name": "Labrador City", "Province Name": "NewfoundLand", "ID": "260" },
  { "City Name": "Cornoerbrook", "Province Name": "NewfoundLand", "ID": "493" },
  { "City Name": "St John", "Province Name": "NewfoundLand", "ID": "464" },
  { "City Name": "Cambridge Bay", "Province Name": "Nunavut", "ID": "254" },
  { "City Name": "Arviat", "Province Name": "Nunavut", "ID": "248" },
  { "City Name": "Iqaluit", "Province Name": "Nunavut", "ID": "246" },
  { "City Name": "Dawson", "Province Name": "Yukon", "ID": "58" },
  { "City Name": "Whitehorse", "Province Name": "Yukon", "ID": "467" },
  { "City Name": "Sept-iles", "Province Name": "Quebec", "ID": "303" },
  { "City Name": "Val-d'Or", "Province Name": "Quebec", "ID": "297" },
  { "City Name": "Inukjuak", "Province Name": "Quebec", "ID": "145" },
  { "City Name": "Montreal", "Province Name": "Quebec", "ID": "450" },
  { "City Name": "Quebec", "Province Name": "Quebec", "ID": "460" },
  { "City Name": "Summerside", "Province Name": "PEI", "ID": "262" },
  { "City Name": "Charlottetown", "Province Name": "PEI", "ID": "462" },
  { "City Name": "Truro", "Province Name": "Nova Scotia", "ID": "265" },
  { "City Name": "Halifax", "Province Name": "Nova Scotia", "ID": "463" },
  { "City Name": "Sydney", "Province Name": "Nova Scotia", "ID": "391" },
  { "City Name": "Inuvik", "Province Name": "Northwest Territories", "ID": "253" },
  { "City Name": "Yellowknife", "Province Name": "Northwest Territories", "ID": "468" },
  { "City Name": "Hay River", "Province Name": "Northwest Territories", "ID": "251" }
]

def _extract_annual_mean_temperature():
  variable = "meantemp"

  for city in cities:
    city_id = city["ID"]
    city_name = city["City Name"]
    province_name = city["Province Name"]

    annual_mean_temp = _download_climate_atlas_data(city_id, variable)

    yield (
      city_id,
      city_name,
      province_name,
      annual_mean_temp
    )

def _load_annual_mean_temperature():
  annual_mean_temperature = _extract_annual_mean_temperature()

  new_dir_xlsx = f"{path}/Data"
  pathlib.Path(new_dir_xlsx).mkdir(parents=True, exist_ok=True)

  writer = pd.ExcelWriter(f'{new_dir_xlsx}/Mean Temperature - Climate Atlas Of Canada.xlsx', engine='xlsxwriter')
  workbook = writer.book

  for (city_id,
       city_name,
       province_name,
       annual_mean_temp) in _extract_annual_mean_temperature():

    worksheet = workbook.add_worksheet(city_name)
    writer.sheets[city_name] = worksheet
    annual_mean_temp.to_excel(writer, sheet_name=city_name)

  writer.close()

_load_annual_mean_temperature()

## Canadian Government Reports

# Annual Precipitation Pipeline

## Downloading Data Utilities

### Climate Atlas Of Canada

I'm not proud of this method... however, I find it as an easier temporary solution...

Rather then download and query data from the PCIC, I will directly download data from the Climate Atlas of Canada.

I don't like doing this. However, given the time constraints, I certaintly need to get this done ASAP.

A major TODO is to use PCIC data to collect the following data. I don't think it's right to scrape data that someone else preprocessed.

In [ ]:
def _download_climate_atlas_data(city_num: int, variable: str):
  possible_values = {"precip", "meantemp"}
  if not variable in possible_values:
    raise ValueError(f"Invalid `variable` value. Should be in {possible_values}")
  
  url = f"https://data.climateatlas.ca/csv/city_{city_num}/BCCAQv2/85/city_{city_num}_BCCAQv2_RCP85_annual_{variable}_ensemble.csv"

  if r.get(url).status_code == 404:
    raise ValueError("Invalid `city_num` value.")

  # The first 7 lines of any Climate Atlas Data Sheet looks
  # like the following (for example)
  #
  # 1 | Bias  correction  method:  BCCAQv2
  # 2 | Model  name:  Ensemble
  # 3 | RCP:  85
  # 4 | Variable:  tasmax  and  tasmin
  # 5 | Season:  annual
  # 6 | 1950-01-01  to  2095-12-31
  # 7 | city_445:  Average  value  Mean  Ensemble
  #
  # I will skip these lines in order to get the needed
  # data
  return pd.read_csv(url, skiprows=7)

## Climate Atlas Of Canada

In [ ]:
# TODO: Specify the ID as a part of the data from the Climate Atlas of Canada.
# the rough part would be tediously renaming everything. But alas, it would
# clarify the distinction between the various data sources.
cities = [
  { "City Name": "Timmins", "Province Name": "Ontario", "ID": "430" },
  { "City Name": "Ottawa", "Province Name": "Ontario", "ID": "459" },
  { "City Name": "Toronto", "Province Name": "Ontario", "ID": "458" },
  { "City Name": "Thunder Bay", "Province Name": "Ontario", "ID": "444" },
  { "City Name": "Regina", "Province Name": "Saskatchewan", "ID": "457" },
  { "City Name": "Saskatoon", "Province Name": "Saskatchewan", "ID": "445" },
  { "City Name": "Thompson", "Province Name": "Manitoba", "ID": "343" },
  { "City Name": "Brandon", "Province Name": "Manitoba", "ID": "430" },
  { "City Name": "Winnipeg", "Province Name": "Manitoba", "ID": "465" },
  { "City Name": "Prince George", "Province Name": "British Columbia", "ID": "443" },
  { "City Name": "Kelowna", "Province Name": "British Columbia", "ID": "442" },
  { "City Name": "Vancouver", "Province Name": "British Columbia", "ID": "455" },
  { "City Name": "Grande Prairie", "Province Name": "Alberta", "ID": "436" },
  { "City Name": "Edmonton", "Province Name": "Alberta", "ID": "466" },
  { "City Name": "Fort McMurray", "Province Name": "Alberta", "ID": "437" },
  { "City Name": "Calgary", "Province Name": "Alberta", "ID": "454" },
  { "City Name": "Edmundston", "Province Name": "New Brunswick", "ID": "273" },
  { "City Name": "Bathurst", "Province Name": "New Brunswick", "ID": "274" },
  { "City Name": "Frecdricton", "Province Name": "New Brunswick", "ID": "461" },
  { "City Name": "Moncton", "Province Name": "New Brunswick", "ID": "394" },
  { "City Name": "Saint John", "Province Name": "New Brunswick", "ID": "393" },
  { "City Name": "Labrador City", "Province Name": "NewfoundLand", "ID": "260" },
  { "City Name": "Cornoerbrook", "Province Name": "NewfoundLand", "ID": "493" },
  { "City Name": "St John", "Province Name": "NewfoundLand", "ID": "464" },
  { "City Name": "Cambridge Bay", "Province Name": "Nunavut", "ID": "254" },
  { "City Name": "Arviat", "Province Name": "Nunavut", "ID": "248" },
  { "City Name": "Iqaluit", "Province Name": "Nunavut", "ID": "246" },
  { "City Name": "Dawson", "Province Name": "Yukon", "ID": "58" },
  { "City Name": "Whitehorse", "Province Name": "Yukon", "ID": "467" },
  { "City Name": "Sept-iles", "Province Name": "Quebec", "ID": "303" },
  { "City Name": "Val-d'Or", "Province Name": "Quebec", "ID": "297" },
  { "City Name": "Inukjuak", "Province Name": "Quebec", "ID": "145" },
  { "City Name": "Montreal", "Province Name": "Quebec", "ID": "450" },
  { "City Name": "Quebec", "Province Name": "Quebec", "ID": "460" },
  { "City Name": "Summerside", "Province Name": "PEI", "ID": "262" },
  { "City Name": "Charlottetown", "Province Name": "PEI", "ID": "462" },
  { "City Name": "Truro", "Province Name": "Nova Scotia", "ID": "265" },
  { "City Name": "Halifax", "Province Name": "Nova Scotia", "ID": "463" },
  { "City Name": "Sydney", "Province Name": "Nova Scotia", "ID": "391" },
  { "City Name": "Inuvik", "Province Name": "Northwest Territories", "ID": "253" },
  { "City Name": "Yellowknife", "Province Name": "Northwest Territories", "ID": "468" },
  { "City Name": "Hay River", "Province Name": "Northwest Territories", "ID": "251" }
]

def _extract_annual_precipitation():
  variable = "precip"

  for city in cities:
    city_id = city["ID"]
    city_name = city["City Name"]
    province_name = city["Province Name"]

    annual_precipitation = _download_climate_atlas_data(city_id, variable)

    yield (
      city_id,
      city_name,
      province_name,
      annual_precipitation
    )

def _load_annual_precipitation():
  annual_precipitation = _extract_annual_precipitation()

  new_dir_xlsx = f"{path}/Data"
  pathlib.Path(new_dir_xlsx).mkdir(parents=True, exist_ok=True)

  writer = pd.ExcelWriter(f'{new_dir_xlsx}/Annual Precipitation Pipeline - Climate Atlas Of Canada.xlsx', engine='xlsxwriter')
  workbook = writer.book

  for (city_id,
       city_name,
       province_name,
       annual_precipitation) in _extract_annual_precipitation():

    worksheet = workbook.add_worksheet(city_name)
    writer.sheets[city_name] = worksheet
    annual_precipitation.to_excel(writer, sheet_name=city_name)

  writer.close()

_load_annual_precipitation()

# Greenhouse Gas Emissions Pipeline

## Statistics Canada

In [ ]:
def _extract_greenhouse_gas_emissions():
  url = "https://www150.statcan.gc.ca/n1/en/tbl/csv/38100097-eng.zip"

  req = r.get(url, stream=True)
  req_stream = BytesIO(req.content)

  zipped_dataset = ZipFile(req_stream)

  greenhouse_gas_emissions = pd.read_csv(zipped_dataset.open('38100097.csv'))

  return greenhouse_gas_emissions

def _transform_greenhouse_gas_emissions():
  greenhouse_gas_emissions = _extract_greenhouse_gas_emissions()
  greenhouse_gas_emissions = greenhouse_gas_emissions[["REF_DATE", "GEO", "VALUE"]]

  greenhouse_gas_emissions["VALUE"] = greenhouse_gas_emissions["VALUE"].fillna(0.0)

  return greenhouse_gas_emissions

def _load_greenhouse_gas_emissions():
  greenhouse_gas_emissions = _transform_greenhouse_gas_emissions()

  new_dir_xlsx = f"{path}/Data"
  pathlib.Path(new_dir_xlsx).mkdir(parents=True, exist_ok=True)

  writer = pd.ExcelWriter(f'{new_dir_xlsx}/Greenhouse Gas Emissions - Statistics Canada.xlsx', engine='xlsxwriter')
  workbook = writer.book

  for province_name, province_df in greenhouse_gas_emissions.groupby("GEO"):
    worksheet = workbook.add_worksheet(province_name)
    writer.sheets[province_name] = worksheet
    province_df.to_excel(writer, sheet_name=province_name)

  writer.close()

_load_greenhouse_gas_emissions()

# Generating Mockups

These set of utilities generates mockups from the ingested data.

## Empty Mockups

In [ ]:
# Config
lines_of_data = 0

# Remove the 'Mocks' directory if it exists
shutil.rmtree(f"{path}/Mocks", ignore_errors=True)

# Get all XLSX files recursively under the 'Data' directory
excel_files = glob.glob(f"{path}/Data/**/*.xlsx", recursive=True)

# Remove the 'Mocks' directory if it exists
shutil.rmtree(f"{path}/Mocks", ignore_errors=True)

# Get all XLSX files recursively under the 'Data' directory
excel_files = glob.glob(f"{path}/Data/**/*.xlsx", recursive=True)

for excel_file in excel_files:
  mock_file = excel_file \
    .replace("/WIP Thanussian/Data", "/WIP Thanussian/Mocks")

  pathlib.Path(mock_file) \
    .parent \
    .mkdir(exist_ok=True, parents=True)

  dfs = pd.read_excel(excel_file, sheet_name=None)

  writer = pd.ExcelWriter(mock_file, engine='xlsxwriter')
  workbook = writer.book

  for sheet_name in dfs:
    worksheet = workbook.add_worksheet(sheet_name)
    writer.sheets[sheet_name] = worksheet

    if lines_of_data == 0:
      df_columns = list(dfs[sheet_name].iloc[0:0])
      df_columns[0] = ""
      df = pd.DataFrame(columns=df_columns, index=range(3))

    elif lines_of_data > 0:
      df = dfs[sheet_name].head(lines_of_data)

    df.to_excel(writer, sheet_name=sheet_name)

  writer.close()

NameError: ignored